Imports

In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler, AutoModelForCausalLM
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
import evaluate


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /home/arjun/NewPytorchEnv/bin/python3.10: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizer

/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('0'), PosixPath('1')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('local/PC'), PosixPath('@/tmp/.ICE-unix/1592,unix/PC')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path wer

Dataset preparation and tokenising

In [2]:
dataset = load_dataset("tatsu-lab/alpaca")

Using custom data configuration tatsu-lab--alpaca-2b32f0433506ef5f
Found cached dataset parquet (/home/arjun/.cache/huggingface/datasets/tatsu-lab___parquet/tatsu-lab--alpaca-2b32f0433506ef5f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset['train'].features

{'instruction': Value(dtype='string', id=None),
 'input': Value(dtype='string', id=None),
 'output': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [4]:
dataset = dataset['train']
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [5]:
# Making dataset smaller for fast training
dataset = dataset.select(range(1000))

In [6]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

new_dataset = {'labels': []}

for example in dataset:
    input_text = example['text'].replace('\n', '')
    input_list = input_text.split('###')
    input_ids = tokenizer(input_list[0] + input_list[1] + input_list[2], truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    # output_id s = tokenizer(input_list[2], truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    new_dataset['labels'].append(input_ids['input_ids'])
    # new_dataset['output'].append(output_ids['input_ids'])

new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format("torch")

In [7]:
new_dataset.features

{'labels': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None)}

DataLoader

In [8]:
dataloader = DataLoader(new_dataset,shuffle=True, batch_size=8)

Loading model

In [9]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

Optimiser and scheduler

In [10]:
num_epochs = 10

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [11]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# print(device)
device = torch.device("cpu")
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [12]:
import torch
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [13]:
progress_bar = tqdm(range(num_training_steps),desc='Training', unit='steps')

model.train()   # Some layers behave differently to training and inference. This sets all those 
                 # layers into training mode
for epoch in range(num_epochs):
    for batch in dataloader:
        batch_data = batch['labels'].to(device)
        batch_data = batch_data.flatten()  # Flatten the tensor to 1-dimensional
        outputs = model.generate(batch_data.unsqueeze(0))
        loss = outputs.loss      # compute loss 
        loss.backward()          # computes gradients
        optimizer.step()         # optimises
        lr_scheduler.step()      # updates lr according to schedule. Improves performance
        optimizer.zero_grad()    # resets the gradients
        progress_bar.update(1)   # updates progress bar by 1

Training:   0%|          | 0/1250 [00:00<?, ?steps/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1024, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


IndexError: index out of range in self

In [14]:
# Progress bar for training
progress_bar = tqdm(range(num_training_steps), desc='Training', unit='steps')

model.train()

for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = (input_ids != tokenizer.pad_token_id).to(device)
        labels = input_ids.clone().to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


Training:   0%|          | 0/1250 [00:00<?, ?steps/s]

KeyError: 'input_ids'